In [2]:
!pip install keras
!pip install torch
!pip install torchtext
!pip install mosestokenizer

# Imports

In [1]:
import pandas as pd
import numpy as np
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchtext
from mosestokenizer import *
import gensim
from tqdm import tqdm
import collections
import math

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load Data

In [2]:
sequences = pd.read_csv('../data/data.csv',nrows=100000).values.flatten()
sequences[:5]

array(['ძეგლთა დაცვის ეროვნულ სააგენტოში აცხადებენ  რომ მათ მიერ  სამხრეთის კარიბჭესთვის  მოწოდებული ინფორმაცია  იმის შესახებ რომ ახალციხის არქივის ეზოში აღმოჩენილი შენობა საკულტო ნაგებობა იყო არასწორია და საბოლოოდ დადგინდა  რომ აქ ტრადიციული მესხური საცხოვრებელი იყო',
       'საქართველოს კულტურული მემკვიდრეობისა და ძეგლთა დაცვის ეროვნულ სააგენტოში აცხადებენ რომ მათ მიერ მოწოდებული პირველადი ინფორმაცია ახალციხის არქივის ეზოში აღმოჩენის შესახებ ნაჩქარევი იყო',
       ' გაირკვა  რომ აქ არც მეჩეთი და არც ეკლესია არ ყოფილა  ჩვეულებრივი საყოფაცხოვრებო დანიშნულების შენობა  ტრადიციული მესხური საცხოვრისი იყო     გვითხრეს კულტურული მემკვიდრეობისა და ძეგლთა დაცვის ეროვნული სააგენტოს საზოგადოებასთან ურთიერთობის სამსახურში',
       'კვლევის ავტორი არ გამორიცხავს  რომ აქ სამხედრო  ან თუნდაც რელიგიური დანიშნულების ობიექტიც ყოფილიყო',
       'შეკითხვაზე  რამდენად შესაძლებელია რომ ეს რელიგიური დანიშნულების ობიექტი მეჩეთი ყოფილიყო  რომლის ნანგრევებზეც ტაძარი აშენდა  ჩოხელი გვპასუხობს '],
      dtype=ob

In [3]:
nrows = len(sequences)
nrows

100000

In [4]:
train = sequences[:int(nrows*0.9)]
test = sequences[int(nrows*0.9):]

In [5]:
avg_sequence_len = (sum([len(x.split()) for x in sequences]) // len(sequences))
avg_sequence_len

10

# Words To Index

In [6]:
# end of sentence <eos>
word_counts = collections.Counter()
word_counts['<pad>'] = 1

for sentence in train:
    for word in sentence.split():
        word_counts[word] += 1
        
unique_words = sorted(word_counts, key=word_counts.get, reverse=True)
vocab_size = len(unique_words) + 1
vocab_size

139769

In [7]:
index_to_word = {index: word for index, word in enumerate(unique_words)}
word_to_index = {word: index for index, word in enumerate(unique_words)}

for i in range(len(train)):
    train[i] = [word_to_index[w] for w in train[i].split()]

Train დატა ინდექსებზეა გადაყვანილი, ახლა შევქმნათ n-gram-ები

In [13]:
# make n-grams
input_train = []
pad_index = word_to_index['<pad>']
n_gram_size = avg_sequence_len // 2

def pad(x):
    if len(x) < avg_sequence_len:
            x += [pad_index] * (avg_sequence_len - len(x))
    return x

for line in train:
    pad(line)
    for i in range(len(line) - n_gram_size):
        n_gram_sequence = line[i:i+n_gram_size]
        input_train.append(n_gram_sequence)

# Embeddings

In [8]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load('../resources/georgian_word2vec.model')
w2v = w2v_model.wv

In [19]:
print(w2v['კაცი'])

[ 7.08971679e-01 -1.51449752e+00 -8.97826731e-01 -9.29931104e-02
 -3.34778619e+00 -8.85849833e-01 -6.82361305e-01  1.93264365e-01
  1.97973764e+00 -1.07326366e-01  3.75844568e-01 -6.44773543e-01
  1.80105960e+00  2.66361976e+00 -1.42565429e-01 -8.48135114e-01
 -4.20123070e-01 -6.38693333e-01 -8.47258493e-02 -4.58429670e+00
  2.47888422e+00 -4.70472813e-01 -1.83740878e+00 -1.54331994e+00
  2.82247686e+00  6.75757647e-01  5.92679381e-01 -5.76169007e-02
 -4.95996892e-01  1.02046371e+00  3.27344686e-02  2.66266316e-01
  1.92116630e+00  2.80386899e-02 -8.97139549e-01 -4.46181923e-01
 -2.34353319e-01  1.40998244e+00  2.35253856e-01 -1.94278586e+00
  3.92591208e-01  2.01633668e+00  3.61283004e-01  2.23645076e-01
  1.81451428e+00  6.71006590e-02 -4.94771689e-01  1.72819376e-01
  7.52462447e-01  1.20438778e+00 -1.84228584e-01  1.71239960e+00
  5.51109135e-01 -1.54601228e+00  5.62165737e-01  1.66284513e+00
 -4.30989981e-01  1.80961800e+00  1.11002803e+00  3.20753306e-01
 -6.98449314e-01 -7.56222

Index to Embeddings

In [9]:
index_to_embedings = {}

for idx, token in index_to_word.items():
    try:
        index_to_embedings[idx] = w2v[token]
    except:
        index_to_embedings[idx] = torch.zeros(100)

In [10]:
print(index_to_embedings[word_to_index['კაცი']])

[ 7.08971679e-01 -1.51449752e+00 -8.97826731e-01 -9.29931104e-02
 -3.34778619e+00 -8.85849833e-01 -6.82361305e-01  1.93264365e-01
  1.97973764e+00 -1.07326366e-01  3.75844568e-01 -6.44773543e-01
  1.80105960e+00  2.66361976e+00 -1.42565429e-01 -8.48135114e-01
 -4.20123070e-01 -6.38693333e-01 -8.47258493e-02 -4.58429670e+00
  2.47888422e+00 -4.70472813e-01 -1.83740878e+00 -1.54331994e+00
  2.82247686e+00  6.75757647e-01  5.92679381e-01 -5.76169007e-02
 -4.95996892e-01  1.02046371e+00  3.27344686e-02  2.66266316e-01
  1.92116630e+00  2.80386899e-02 -8.97139549e-01 -4.46181923e-01
 -2.34353319e-01  1.40998244e+00  2.35253856e-01 -1.94278586e+00
  3.92591208e-01  2.01633668e+00  3.61283004e-01  2.23645076e-01
  1.81451428e+00  6.71006590e-02 -4.94771689e-01  1.72819376e-01
  7.52462447e-01  1.20438778e+00 -1.84228584e-01  1.71239960e+00
  5.51109135e-01 -1.54601228e+00  5.62165737e-01  1.66284513e+00
 -4.30989981e-01  1.80961800e+00  1.11002803e+00  3.20753306e-01
 -6.98449314e-01 -7.56222

In [16]:
embedding_vectors = [x for _, x in index_to_embedings.items()]

# DataLoader 

In [23]:
input_train[:5]

[[4189, 342, 651, 5760, 1955],
 [342, 651, 5760, 1955, 2],
 [651, 5760, 1955, 2, 37],
 [5760, 1955, 2, 37, 46],
 [1955, 2, 37, 46, 2463]]

In [24]:
len(input_train)

732735

In [25]:
def index_to_vec(index):
    word = index_to_word[index]
    try:
        return torch.tensor(w2v[word])
    except:
        return torch.zeros(100)
    
# pad or strip data, also add <eos>
def collate_batch(batch):
    label_list, text_list = [], []
    for x in batch:
        label_list.append(x[1:])
        x = x[:-1]
        text_list.append(torch.Tensor(x).long())
    text_list = torch.stack(text_list, dim=0)
    label_list = torch.Tensor(label_list).long()
    
    return text_list, label_list

In [26]:
batch_size=128

train_dl = DataLoader(input_train[:int(len(input_train)*0.7)], 
                      batch_size=batch_size, 
                      shuffle=True, 
                      collate_fn=collate_batch)

valid_dl = DataLoader(input_train[int(len(input_train)*0.7):], 
                      batch_size=batch_size, 
                      shuffle=False, 
                      collate_fn=collate_batch)

# Model

In [11]:
class LSTM(nn.Module):
    def __init__(self,*, n_vocab, num_layers, embedding_dim, hidden_size):
        super(LSTM, self).__init__()
        self.n_vocab = n_vocab
        self.num_layers = num_layers
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        
        self.emb = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_vectors), 
            freeze=False
        ) 
        
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True,
            dropout=0.3,
        )

        self.fc = nn.Linear(self.hidden_size, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        x = self.emb(x)
        output, state = self.lstm(x, (h0, c0))
        logits = self.fc(output)
        return logits, state

# Train

In [14]:
num_layers=2
embedding_dim=100
hidden_size=128
seq_len = n_gram_size-1

In [21]:
model = LSTM(
    n_vocab=vocab_size,
    num_layers=num_layers,
    embedding_dim = embedding_dim,
    hidden_size= hidden_size
)

model.train()

dataloader = train_dl
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, min_lr=1e-6, patience=10) 

for epoch in range(1):

    for batch, (x, y) in enumerate(dataloader):
        optimizer.zero_grad()

        y_pred, (state_h, state_c) = model(x)
        loss = criterion(y_pred.transpose(1, 2), y)
        perplexity = math.exp(loss)

        loss.backward()
        optimizer.step()
        if(batch % 10 == 0):
            print({ 'epoch': epoch, 'batch': batch, 'loss': loss.item(), 'perplexity': perplexity })

{'epoch': 0, 'batch': 0, 'loss': 11.860724449157715, 'perplexity': 141594.7593274818}
{'epoch': 0, 'batch': 10, 'loss': 11.800698280334473, 'perplexity': 133345.43293729678}
{'epoch': 0, 'batch': 20, 'loss': 11.485949516296387, 'perplexity': 97338.4652434557}
{'epoch': 0, 'batch': 30, 'loss': 10.450186729431152, 'perplexity': 34550.8257629988}
{'epoch': 0, 'batch': 40, 'loss': 9.45033073425293, 'perplexity': 12712.368984329485}
{'epoch': 0, 'batch': 50, 'loss': 9.441768646240234, 'perplexity': 12603.989202952747}
{'epoch': 0, 'batch': 60, 'loss': 9.130753517150879, 'perplexity': 9234.978062445525}
{'epoch': 0, 'batch': 70, 'loss': 9.256898880004883, 'perplexity': 10476.593737601084}
{'epoch': 0, 'batch': 80, 'loss': 9.193681716918945, 'perplexity': 9834.793330423528}
{'epoch': 0, 'batch': 90, 'loss': 9.114057540893555, 'perplexity': 9082.071105396599}
{'epoch': 0, 'batch': 100, 'loss': 9.513969421386719, 'perplexity': 13547.664114683561}
{'epoch': 0, 'batch': 110, 'loss': 9.10488224029

{'epoch': 0, 'batch': 950, 'loss': 7.726149082183838, 'perplexity': 2266.8558951904915}
{'epoch': 0, 'batch': 960, 'loss': 8.147567749023438, 'perplexity': 3454.965494471167}
{'epoch': 0, 'batch': 970, 'loss': 8.273262977600098, 'perplexity': 3917.7115639141994}
{'epoch': 0, 'batch': 980, 'loss': 8.744501113891602, 'perplexity': 6276.081588773073}
{'epoch': 0, 'batch': 990, 'loss': 8.36485481262207, 'perplexity': 4293.488332470524}
{'epoch': 0, 'batch': 1000, 'loss': 8.096378326416016, 'perplexity': 3282.558167200096}
{'epoch': 0, 'batch': 1010, 'loss': 8.31877613067627, 'perplexity': 4100.138902086716}
{'epoch': 0, 'batch': 1020, 'loss': 8.293807983398438, 'perplexity': 3999.0334899721565}
{'epoch': 0, 'batch': 1030, 'loss': 8.320867538452148, 'perplexity': 4108.722937699267}
{'epoch': 0, 'batch': 1040, 'loss': 8.522899627685547, 'perplexity': 5028.613744955827}
{'epoch': 0, 'batch': 1050, 'loss': 7.871315002441406, 'perplexity': 2621.0099575071927}
{'epoch': 0, 'batch': 1060, 'loss':

{'epoch': 0, 'batch': 1880, 'loss': 7.6862945556640625, 'perplexity': 2178.2880642769223}
{'epoch': 0, 'batch': 1890, 'loss': 7.96051025390625, 'perplexity': 2865.5347298253396}
{'epoch': 0, 'batch': 1900, 'loss': 7.747730255126953, 'perplexity': 2316.3090130745527}
{'epoch': 0, 'batch': 1910, 'loss': 7.624538421630859, 'perplexity': 2047.8350104917922}
{'epoch': 0, 'batch': 1920, 'loss': 7.65710973739624, 'perplexity': 2115.6338460719985}
{'epoch': 0, 'batch': 1930, 'loss': 7.879726409912109, 'perplexity': 2643.149321316506}
{'epoch': 0, 'batch': 1940, 'loss': 7.741633415222168, 'perplexity': 2302.2298107850106}
{'epoch': 0, 'batch': 1950, 'loss': 8.12581729888916, 'perplexity': 3380.6297878590794}
{'epoch': 0, 'batch': 1960, 'loss': 7.917121410369873, 'perplexity': 2743.8612161718897}
{'epoch': 0, 'batch': 1970, 'loss': 7.886530876159668, 'perplexity': 2661.1958705977027}
{'epoch': 0, 'batch': 1980, 'loss': 7.641680717468262, 'perplexity': 2083.242217421573}
{'epoch': 0, 'batch': 199

{'epoch': 0, 'batch': 2810, 'loss': 7.635678768157959, 'perplexity': 2070.7761510033774}
{'epoch': 0, 'batch': 2820, 'loss': 8.004242897033691, 'perplexity': 2993.6327546951024}
{'epoch': 0, 'batch': 2830, 'loss': 7.556046009063721, 'perplexity': 1912.2694493292804}
{'epoch': 0, 'batch': 2840, 'loss': 7.331290245056152, 'perplexity': 1527.3511585030103}
{'epoch': 0, 'batch': 2850, 'loss': 7.8025288581848145, 'perplexity': 2446.781724469831}
{'epoch': 0, 'batch': 2860, 'loss': 7.720390796661377, 'perplexity': 2253.840201711723}
{'epoch': 0, 'batch': 2870, 'loss': 7.7013020515441895, 'perplexity': 2211.2252475612563}
{'epoch': 0, 'batch': 2880, 'loss': 7.668887615203857, 'perplexity': 2140.6988394640807}
{'epoch': 0, 'batch': 2890, 'loss': 7.5691752433776855, 'perplexity': 1937.5416221130452}
{'epoch': 0, 'batch': 2900, 'loss': 7.798945426940918, 'perplexity': 2438.0295411804445}
{'epoch': 0, 'batch': 2910, 'loss': 7.586371421813965, 'perplexity': 1971.1480565578527}
{'epoch': 0, 'batch'

{'epoch': 0, 'batch': 3740, 'loss': 7.160329818725586, 'perplexity': 1287.3354502327475}
{'epoch': 0, 'batch': 3750, 'loss': 7.251073837280273, 'perplexity': 1409.6177358374227}
{'epoch': 0, 'batch': 3760, 'loss': 7.13455867767334, 'perplexity': 1254.5831909952242}
{'epoch': 0, 'batch': 3770, 'loss': 7.7371931076049805, 'perplexity': 2292.0298644282057}
{'epoch': 0, 'batch': 3780, 'loss': 6.870387077331543, 'perplexity': 963.321373521333}
{'epoch': 0, 'batch': 3790, 'loss': 7.478954315185547, 'perplexity': 1770.3885392618868}
{'epoch': 0, 'batch': 3800, 'loss': 7.35610294342041, 'perplexity': 1565.72294722852}
{'epoch': 0, 'batch': 3810, 'loss': 7.152223587036133, 'perplexity': 1276.9421928000359}
{'epoch': 0, 'batch': 3820, 'loss': 7.289883136749268, 'perplexity': 1465.399435855405}
{'epoch': 0, 'batch': 3830, 'loss': 7.3074541091918945, 'perplexity': 1491.3754727598803}
{'epoch': 0, 'batch': 3840, 'loss': 7.264123916625977, 'perplexity': 1428.1339151601298}
{'epoch': 0, 'batch': 3850

# Save And Load Model

In [22]:
torch.save(model.state_dict(), '../resources/nn_lstm_model_state')

In [17]:
model = LSTM(
    n_vocab=vocab_size,
    num_layers=2,
    embedding_dim = 100,
    hidden_size= 128
)
model.load_state_dict(torch.load('../resources/nn_lstm_model_state'))

<All keys matched successfully>

# Save Trained Embeddings

In [39]:
trained_embeddings = model.state_dict()['emb.weight']
trained_embeddings.shape

torch.Size([139768, 100])

In [42]:
with open('../resources/trained_word2vec_embeddings', 'wb') as fout:
    fout.write(gensim.utils.to_utf8("%s %s\n" % (trained_embeddings.size(0), trained_embeddings.size(1))))
    for word, _ in tqdm(word_counts.most_common()):
        index = word_to_index[word]
        embedding = trained_embeddings[index]
        fout.write(gensim.utils.to_utf8("%s %s\n" % (word, ' '.join(repr(val.item()) for val in embedding))))

100%|██████████| 139768/139768 [51:07<00:00, 45.56it/s] 


# Predict

In [255]:
def word2index(w):
    try:
        return word_to_index[w]
    except:
        return word_to_index['<pad>']


def predict(model, text, next_words=100):
    model.eval()

    words = text.split(' ')

    for i in range(0, next_words):
        x = torch.tensor([[word2index(w) for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x)

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        while(index_to_word[word_index] == '<pad>'):
            word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(index_to_word[word_index])
    return ' '.join(words)

predict(model, 'ბიჭი', 5)

'ბიჭი რის გამოც მიღებული მთავარია მაშინ'

In [263]:
predict(model, 'გოგო შევიდა კაფეში', 7)

'გოგო შევიდა კაფეში დავიწყებ ვერ უყვიროდნენ საკუთარ სიტყვას ღმერთებმა სულიერ'

In [56]:
predict(model, 'მერე რა რომ ზამთარია', 7)

'მერე რა რომ ზამთარია რომელშიც ითქვა შუახნის მწუხრი ველთა სანუგეშოდ და'

In [273]:
predict(model, 'ხვალ', 7)

'ხვალ სადაც ვუჩივლებდი ასევე ნახსენები უნდა ფორმალობად ბოლომდე'

In [253]:
def beam_search(context, model, index_list, num_beams=3):
    if num_beams < 0: 
        return 1, -1
    y_pred, (state_h, state_c) = model(context)
    last_word_logits = y_pred[0][-1]
    predictions = torch.nn.functional.softmax(last_word_logits, dim=0).detach()
    top_words = torch.topk(predictions, dim=-1, k=10)
    indices = top_words.indices
    values = top_words.values
    top_results = []
    for indice, pr in zip(indices, values):
        l = context.tolist()[0].copy()
        l.append(indice.item())
        l = l[-10:]
        new_context = torch.tensor(l).unsqueeze(dim=0)
        prob, word_index = beam_search(new_context, model, index_list, num_beams=num_beams-1)
        prob = prob * pr.item()
        top_results.append((prob, indice.item()))
    top_results = sorted(top_results, key=lambda tup: tup[0])
    
    for prob, word_index in top_results:
        if word_index not in index_list:
            return prob, word_index

In [254]:
def generate_with_beamsearch(model, context, next_words):
    model.eval()

    words = context.split(' ')
    index_list = [word_to_index['<pad>']]
    
    for i in range(0, next_words):
        ind = ([word2index(w) for w in words[i:]][0])
        index_list.append(ind)
        x = torch.tensor([[word2index(w) for w in words[i:]]])
        _, prediction_indice = beam_search(x, model, index_list)
        words.append(index_to_word[prediction_indice])
    return ' '.join(words)

generate_with_beamsearch(model, 'გუშინ', 8)

'გუშინ საქართველოს პრეზიდენტის და სხვა დროს ამ უფლება იმისა'

In [252]:
generate_with_beamsearch(model, 'დღეს', 8)

'დღეს როდესაც ყველა მნიშვნელოვანი მთავარი ინფორმაციისთვის რომ მისი მხრიდან'

In [247]:
generate_with_beamsearch(model, 'არასოდეს', 6)

'არასოდეს ვიცი კი ვერ ახსენებ სიმპათიას თუ'